# Animate json files of Laxmi facing the camera.

## Import packages.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.animation as animation
from matplotlib.ticker import MultipleLocator
from IPython.display import HTML

## Specify folder of .json files generated by OpenPose.

In [ ]:
zdir = 'J:/Hack-2502-Devpost-QualcommSnapdragon/7PyLaxmiFace/d_json/'
zpfx = '4'

## Show text of the first json file from the video.

- Each json file refers to multiple persons. But, this analysis is only for the first person.

In [ ]:
with open(zdir + zpfx + '_000000000000_keypoints.json', 'r') as zfil:
    zful = zfil.read().replace('\n', '').replace(" ","")
zful 

## Extract coordinates of 15 keypoints of the first person.

- Keypoints extracted: Nose, Neck, RShoulder, RElbow, RWrist, LShoulder, LElbow, LWrist, MidHip, RHip, RKnee, RAnkle, LHip, LKnee, LAnkle

- Keypoints not extracted: REye, LEye, REar, LEar, LBigToe, LSmallToe, LHeel, RBigToe, RSmallToe, RHeel, Background

In [ ]:
zbeg = zful.find('2d":[') + 6
zend = zful.find('],"face')
zstr = zful[zbeg:zend]
zstr

## Arrange keypoints for creating a wireframe through Matplotlib.

In [ ]:
zwir = [1,2,3,4,5,4,3,2,6,7,8,7,6,2,9,10,11,12,11,10,9,13,14,15]

## Define a function for creating two arrays of x and y coordinates.

In [ ]:
def fpts(p):
    with open(zdir + zpfx + '_000000000' + str(9999000+p).replace('9999','') + '_keypoints.json', 'r') as zfil:
        zful = zfil.read().replace('\n', '').replace(" ","")
    zbeg = zful.find('2d":[') + 5
    zend = zful.find('],"face')
    zstr = zful[zbeg:zend]
    zarr = zstr.split(',')
    zxps = [int(float(zarr[3*m-3])) for m in zwir]
    zyps = [int(float(zarr[3*m-2])) for m in zwir]
    return zxps, zyps

## Create two arrays of x and y coordinates from all json files.

- Ignore records not belonging to the first peson, by using criteria of locations of the keypoints.

In [ ]:
zxar = []
zyar = []
zlen = []

zxp0, zyp0 = fpts(0)

for i in range(205):
    zxps, zyps = fpts(i)
    if (zxps[0]>0) and (zyps[23]-zyps[0]>250) and (zyps[23]-zyps[0]<300):
        zxar = zxar + [[zxps[j]-zxps[0] for j in range(24)]]
        zyar = zyar + [[-2*zyps[j]+2*zyps[0] for j in range(24)]]
        zlen = zlen + [zyps[23]-zyps[0]]
        zxp0 = zxps[0]
        zyp0 = zyps[0]

In [ ]:
print("Number of json records accepted for animation:")
print(len(zxar))

## Evaluate correctness of raising the feet.

In [ ]:
print("Nose to ankle distances in accepted records:")
print(zlen)

In [ ]:
zfit = int(200*(max(zlen)-min(zlen))/(max(zlen)+min(zlen)))
zfit

In [ ]:
if zfit < 5:
    print("Person is raising feet less than appropriate.")
else:
    if zfit < 15:
        print("Person is raising feet correctly.")
    else:
        if zfit < 20:
            print("Person is raising feet more than appropriate.")
        else:        
            print("Person seems to be jumping or jerking while walking.")

## Create an animation file and html using Matplotlib.

In [ ]:
%%capture
fig, ax = plt.subplots(figsize=(4,8))
plt.xticks([-100,150,150,-100])
plt.yticks([50,50,-600,-600])
x = []
y = []
plt.plot(x, y, "green", linestyle='-')
plt.text(0,0,"x", font="Times New Roman")

z, = ax.plot(x, y, 'go')

def update(p):
    z.set_data(zxar[p], zyar[p])
    z.set_linestyle('-')
    return z

anim = animation.FuncAnimation(fig, update, frames=range(len(zxar))) # , fargs=(z), interval=1000)
anim.save(zdir + '0.gif', writer='Pillow', fps=15)
zhtm = HTML(anim.to_jshtml())

plt.show()

## Display the animation through html.

In [ ]:
zhtm